# <b><span style='color:#2ae4f5'>|</span> Scenario Generation <a id = "4" > </b> 

This notebook implements a scenario generation technique for the farmer problem and applies a backward reduction approach for scenario reduction.

In [2]:
import numpy as np
from pulp import *
import random
import math
from scipy.stats import norm

## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>|</span></b> Inverse Transform Sampling <a id = "5" ></b></p>
</div>

The problem that the inverse transform sampling method solves is as follows:

- Let $X$ be a random variable whose distribution can be described by the cumulative distribution function $F_x$

- We want to generate values of $X$ which are distributed according to this distribution.

The inverse transform sampling method works as follows:

- **Step 1**: Generate a random number u from the standard uniform distribution in the interval [0, 1] i.e. from $U \in [0, 1]$.

- **Step 2**: Find the generalized inverse of the desired CDF, $F^{-1}_{X}(U)$

Ref: https://en.wikipedia.org/wiki/Inverse_transform_sampling


Normal Distribution: $F^{-1}(p) = \mu + \sqrt2 \sigma + erf^{-1}(2p-1)$

In [3]:
def inverse_cdf_normal(mean, std_dev, p):
    return norm.ppf(p, loc=mean, scale=std_dev)

In [4]:
n = 100
miu_wheat, sigma_wheat = 2.5, 0.5
miu_corn, sigma_corn = 3, 0.6
miu_sugar, sigma_sugar = 20, 4
num_of_kisi = 3

random_list = np.zeros(n - 1)
scenarios = np.zeros((n, num_of_kisi))
probability = np.zeros(n)

# Generating random numbers for n scenarios
for i in range(n - 1):
	random_list[i] = random.random()

sorted_numbers = np.sort(random_list)

# Generating scenarios from randomNumbers and assign its probability
for i, prob in enumerate(sorted_numbers):
	if i == 0:
		probability[i] = prob
		scenarios[i][0] = inverse_cdf_normal(mean=miu_wheat, std_dev=sigma_wheat, p=prob/2)
		scenarios[i][1] = inverse_cdf_normal(mean=miu_corn, std_dev=sigma_corn, p=prob/2)
		scenarios[i][2] = inverse_cdf_normal(mean=miu_sugar, std_dev=sigma_sugar, p=prob/2)
  
	elif i == n - 2:
		probability[i] = (sorted_numbers[i] - sorted_numbers[i - 1])
		central_prob = (sorted_numbers[i] + sorted_numbers[i - 1]) / 2
		scenarios[i][0] = inverse_cdf_normal(mean=miu_wheat, std_dev=sigma_wheat, p=central_prob)
		scenarios[i][1] = inverse_cdf_normal(mean=miu_corn, std_dev=sigma_corn, p=central_prob)
		scenarios[i][2] = inverse_cdf_normal(mean=miu_sugar, std_dev=sigma_sugar, p=central_prob)
		probability[n - 1] = (1 - sorted_numbers[n - 2])
		central_prob = (1 + sorted_numbers[n - 2]) / 2
		scenarios[n - 1][0] = inverse_cdf_normal(mean=miu_wheat, std_dev=sigma_wheat, p=central_prob)
		scenarios[n - 1][1] = inverse_cdf_normal(mean=miu_corn, std_dev=sigma_corn, p=central_prob)
		scenarios[n - 1][2] = inverse_cdf_normal(mean=miu_sugar, std_dev=sigma_sugar, p=central_prob)
  
	else:
		probability[i] = (sorted_numbers[i] - sorted_numbers[i - 1])
		central_prob = (sorted_numbers[i] + sorted_numbers[i - 1]) / 2
		scenarios[i][0] = inverse_cdf_normal(mean=miu_wheat, std_dev=sigma_wheat, p=central_prob)
		scenarios[i][1] = inverse_cdf_normal(mean=miu_corn, std_dev=sigma_corn, p=central_prob)
		scenarios[i][2] = inverse_cdf_normal(mean=miu_sugar, std_dev=sigma_sugar, p=central_prob)

In [5]:
total_scenario, total_probabilities = scenarios, probability

In [6]:
total_scenario, sum(total_probabilities)

(array([[ 1.18663188,  1.42395825,  9.49305501],
        [ 1.34560742,  1.6147289 , 10.76485935],
        [ 1.38996305,  1.66795566, 11.11970443],
        [ 1.43986247,  1.72783496, 11.51889975],
        [ 1.60939639,  1.93127567, 12.87517115],
        [ 1.71150404,  2.05380484, 13.6920323 ],
        [ 1.75178903,  2.10214683, 14.01431221],
        [ 1.81232545,  2.17479054, 14.4986036 ],
        [ 1.84514109,  2.2141693 , 14.76112869],
        [ 1.85981534,  2.23177841, 14.8785227 ],
        [ 1.92131453,  2.30557743, 15.37051622],
        [ 1.98104512,  2.37725414, 15.84836094],
        [ 2.00104495,  2.40125394, 16.00835959],
        [ 2.0281543 ,  2.43378516, 16.2252344 ],
        [ 2.05308638,  2.46370365, 16.42469101],
        [ 2.05727619,  2.46873142, 16.45820949],
        [ 2.05859643,  2.47031572, 16.46877146],
        [ 2.07398495,  2.48878193, 16.59187956],
        [ 2.10054258,  2.52065109, 16.80434063],
        [ 2.11987645,  2.54385173, 16.95901156],
        [ 2.12929397

## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>|</span></b> Scenario Reduction <a id = "6" ></b></p>
</div>

In [7]:
# Creating the index matrices for each scenario
total_index = np.zeros(n)
for i, scenario in enumerate(total_scenario):
    total_index[i] = i    
total_index

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94., 95., 96., 97., 98., 99.])

In [8]:
first_year = {"removed_scenarios": {"scenario": [], "probability": [], "index": []},
              "keeped_scenarios": {"scenario": total_scenario, "probability": total_probabilities, "index": total_index}}

In [9]:
# Define distance matrix to store distance between scenario i & j
minimum_distances = []
distance_matrix = np.zeros((n, n))

# Calculating each indexes of Distance Matrix
for i1, i in enumerate(first_year["keeped_scenarios"]["scenario"]):
	for i2, j in enumerate(first_year["keeped_scenarios"]["scenario"]):
		if i1 != i2:
			# Calcualte distances based on Euclidean Distance
			distance_matrix[i1][i2] = (math.sqrt(math.pow((i[0] - j[0]), 2) + math.pow((i[1] - j[1]), 2) + math.pow((i[2] - j[2]), 2)))
		else:
			distance_matrix[i1][i2] = 10000
   
# print(distance_matrix)

# find smallest (Probability * distance)
for i, distance in enumerate(distance_matrix):
	# Calculating (probability * distance) for each scenario
	minimum_distances.append(first_year["keeped_scenarios"]["probability"][i] * min(distance))

min_value = min(minimum_distances)
min_index = minimum_distances.index(min_value)
first_year["removed_scenarios"]["scenario"].append(first_year["keeped_scenarios"]["scenario"][min_index])
first_year["removed_scenarios"]["probability"].append(first_year["keeped_scenarios"]["probability"][min_index])
first_year["removed_scenarios"]["index"].append(first_year["keeped_scenarios"]["index"][min_index])
first_year["keeped_scenarios"]["scenario"] = np.delete(first_year["keeped_scenarios"]["scenario"], min_index, axis=0)
first_year["keeped_scenarios"]["probability"] = np.delete(first_year["keeped_scenarios"]["probability"], min_index, axis=0)
first_year["keeped_scenarios"]["index"] = np.delete(first_year["keeped_scenarios"]["index"], min_index)

# Define while condition
stop_condition = False
minimum_distances = []

# Stop Condition: # of keeped scenarios <= 15
while not stop_condition:
	total_distances = []
	# Candidate the keeped scenarios
	for i, j in enumerate(first_year["keeped_scenarios"]["scenario"]):
		first_year["removed_scenarios"]["scenario"].append(j)
		first_year["removed_scenarios"]["probability"].append(first_year["keeped_scenarios"]["probability"][i])
		first_year["removed_scenarios"]["index"].append(first_year["keeped_scenarios"]["index"][i])
		np.delete(first_year["keeped_scenarios"]["scenario"], i, 0)
		np.delete(first_year["keeped_scenarios"]["probability"], i, 0)
		keeped_index = np.delete(first_year["keeped_scenarios"]["index"], i, 0)
		# print(keeped_index)
		# Calculating the distance between keeped and removed scenarios
		for i1, k in enumerate(first_year["removed_scenarios"]["index"]):
			distances = []
			for i2, z in enumerate(keeped_index):
				distances.append(distance_matrix[int(k)][int(z)])
			minimum_distances.append(first_year["removed_scenarios"]["probability"][i1] * min(distances))
		# print(minimum_distances)
		total_distances.append(sum(minimum_distances))
		# print(total_distances)
		first_year["removed_scenarios"]["scenario"].pop(-1)
		first_year["removed_scenarios"]["probability"].pop(-1)
		first_year["removed_scenarios"]["index"].pop(-1)
		minimum_distances = []
	# Find the best choice for removing
	min_value = min(total_distances)
	min_index = total_distances.index(min_value)

	# Check the stop condition
	if len(first_year["keeped_scenarios"]["scenario"]) <= 40:
		stop_condition = True
	else:
		first_year["removed_scenarios"]["scenario"].append(first_year["keeped_scenarios"]["scenario"][min_index])
		first_year["removed_scenarios"]["probability"].append(first_year["keeped_scenarios"]["probability"][min_index])
		first_year["removed_scenarios"]["index"].append(first_year["keeped_scenarios"]["index"][min_index])
		first_year["keeped_scenarios"]["scenario"] = np.delete(first_year["keeped_scenarios"]["scenario"], min_index, 0)
		first_year["keeped_scenarios"]["probability"] = np.delete(first_year["keeped_scenarios"]["probability"], min_index, 0)
		first_year["keeped_scenarios"]["index"] = np.delete(first_year["keeped_scenarios"]["index"], min_index, 0)

In [10]:
first_year

{'removed_scenarios': {'scenario': [array([ 2.59897376,  3.11876852, 20.7917901 ]),
   array([ 2.59872598,  3.11847118, 20.78980786]),
   array([ 2.05727619,  2.46873142, 16.45820949]),
   array([ 2.63834774,  3.16601729, 21.10678191]),
   array([ 2.75410896,  3.30493075, 22.03287169]),
   array([ 2.13234924,  2.55881909, 17.05879393]),
   array([ 2.62079567,  3.14495481, 20.96636537]),
   array([ 2.68215248,  3.21858297, 21.45721983]),
   array([ 2.31589404,  2.77907285, 18.52715232]),
   array([ 2.70409881,  3.24491857, 21.63279046]),
   array([ 2.85966269,  3.43159523, 22.87730154]),
   array([ 2.33578857,  2.80294628, 18.68630857]),
   array([ 2.05859643,  2.47031572, 16.46877146]),
   array([ 2.83444738,  3.40133685, 22.67557902]),
   array([ 2.27199218,  2.72639062, 18.17593746]),
   array([ 2.59979914,  3.11975896, 20.79839309]),
   array([ 2.31808613,  2.78170335, 18.54468901]),
   array([ 2.75660033,  3.30792039, 22.05280261]),
   array([ 4.04986258,  4.85983509, 32.39890062])

## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>|</span></b> Clustering <a id = "7" ></b></p>
</div>

In [11]:
sum(first_year['keeped_scenarios']['probability'])

0.7315000222866626

In [12]:
# create dictionary to store central represenatative and its probability
stage_cluster = {"cluster": first_year["keeped_scenarios"]["scenario"],
					"probability": first_year["keeped_scenarios"]["probability"]}

# Find the best represetative for removed scenario
for i1, j in enumerate(first_year["removed_scenarios"]["scenario"]):
	distances = []
	for i2, k in enumerate(first_year["keeped_scenarios"]["scenario"]):
		distance = math.sqrt(math.pow((j[0] - k[0]), 2) + math.pow((j[1] - k[1]), 2) + math.pow((j[2] - k[2]), 2))
		distances.append(distance)
	min_value = min(distances)
	min_index = distances.index(min_value)
	# modifying the probability of each cluster
	stage_cluster["probability"][min_index] += first_year["removed_scenarios"]["probability"][i1]

In [13]:
sum(first_year['keeped_scenarios']['probability'])

1.0

In [14]:
first_year['keeped_scenarios']

{'scenario': array([[ 1.18663188,  1.42395825,  9.49305501],
        [ 1.34560742,  1.6147289 , 10.76485935],
        [ 1.43986247,  1.72783496, 11.51889975],
        [ 1.60939639,  1.93127567, 12.87517115],
        [ 1.75178903,  2.10214683, 14.01431221],
        [ 1.81232545,  2.17479054, 14.4986036 ],
        [ 1.85981534,  2.23177841, 14.8785227 ],
        [ 1.92131453,  2.30557743, 15.37051622],
        [ 1.98104512,  2.37725414, 15.84836094],
        [ 2.0281543 ,  2.43378516, 16.2252344 ],
        [ 2.07398495,  2.48878193, 16.59187956],
        [ 2.10054258,  2.52065109, 16.80434063],
        [ 2.14325901,  2.57191082, 17.14607211],
        [ 2.17463756,  2.60956507, 17.39710049],
        [ 2.21041885,  2.65250262, 17.68335079],
        [ 2.25975425,  2.7117051 , 18.07803398],
        [ 2.29083444,  2.74900132, 18.32667548],
        [ 2.33095888,  2.79715066, 18.64767108],
        [ 2.35809472,  2.82971366, 18.86475774],
        [ 2.40436266,  2.8852352 , 19.23490131],
        